In [126]:
import requests
import pandas as pd
import json


In [127]:
API_KEY = "a2faf0838c2aacce44be67cea8b40c06-48d69bac26199a4d5850294881134c34"
ACCOUNT_ID = "101-004-21509763-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [128]:
session = requests.Session()

In [129]:
session.headers.update({
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {API_KEY}'
})

In [130]:
params = dict(
    count = 10,
    granularity = "H1"
)

In [131]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [132]:
response = session.get(url, params=None, data=None, headers=None)

In [133]:
response.status_code

200

In [134]:
data = response.json()

In [135]:
data.keys()

dict_keys(['instruments', 'lastTransactionID'])

In [136]:
instruments_list = data["instruments"]

In [137]:
len(instruments_list)

123

In [138]:
instruments_list[1].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [139]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [140]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in key_i }

In [141]:
instruments_dict['AU200_AUD']

{'name': 'AU200_AUD',
 'type': 'CFD',
 'displayName': 'Australia 200',
 'pipLocation': 0,
 'displayPrecision': 1,
 'tradeUnitsPrecision': 1,
 'marginRate': '0.05'}

In [142]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [146]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data
    

In [147]:
code, data = fetch_candles('EUR_USD', count=20)

In [148]:
code

404